In [1]:
!pip install wandb ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.2 MB/s eta 0:00:00


In [2]:
import ultralytics
from ultralytics import YOLO
import wandb
from datetime import datetime

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Login to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
wandb.login()

True

In [14]:
!yolo settings wandb=True

✅ Updated 'wandb=True'
JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [9]:
DATA_DIR = "/content/drive/MyDrive/contrails_swg/data"

In [10]:
import json
import os
import shutil
import random
import numpy as np
from pathlib import Path
from tqdm import tqdm

def convert_coco_to_yolo(json_path, source_img_dir, output_dir, val_split=0.2):
    with open(json_path, 'r') as f:
        coco = json.load(f)

    # Create directories
    output_dir = Path(output_dir)
    (output_dir / 'images' / 'train').mkdir(parents=True, exist_ok=True)
    (output_dir / 'images' / 'val').mkdir(parents=True, exist_ok=True)
    (output_dir / 'labels' / 'train').mkdir(parents=True, exist_ok=True)
    (output_dir / 'labels' / 'val').mkdir(parents=True, exist_ok=True)

    # Map categories to indices (0-indexed)
    # Ensure consistent ordering
    categories = sorted(coco['categories'], key=lambda x: x['id'])
    cat_id_to_idx = {cat['id']: i for i, cat in enumerate(categories)}
    cat_names = [cat['name'] for cat in categories]

    print(f"Categories mapping: {cat_id_to_idx}")
    print(f"Category names: {cat_names}")

    # Group annotations by image
    img_to_anns = {}
    for ann in coco['annotations']:
        img_id = ann['image_id']
        if img_id not in img_to_anns:
            img_to_anns[img_id] = []
        img_to_anns[img_id].append(ann)

    # Split images
    images = coco['images']
    random.shuffle(images)
    split_idx = int(len(images) * (1 - val_split))
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    def process_images(image_list, split_name):
        for img_info in tqdm(image_list, desc=f"Processing {split_name}"):
            img_id = img_info['id']
            file_name = img_info['file_name']
            width = img_info['width']
            height = img_info['height']

            # Copy image
            src_path = Path(source_img_dir) / file_name
            if not src_path.exists():
                # Try adding underscore after 8th char (date)
                if len(file_name) > 8 and file_name[8] != '_':
                    alt_name = file_name[:8] + '_' + file_name[8:]
                    src_path_alt = Path(source_img_dir) / alt_name
                    if src_path_alt.exists():
                        src_path = src_path_alt
                        # print(f"Fixed filename: {file_name} -> {alt_name}")

            dst_path = output_dir / 'images' / split_name / file_name # Keep original name in dest or use fixed?
            # Better to use the fixed name in dest so it matches the label file which is derived from file_name
            # But wait, the label file is derived from file_name (from JSON).
            # If I change the image name on disk, I should also change the label filename.
            # Let's use the found src filename as the basis for dest filename to ensure consistency.

            if src_path.exists():
                dst_path = output_dir / 'images' / split_name / src_path.name
                shutil.copy(src_path, dst_path)
                # Update file_name to match the one on disk for label generation
                file_name = src_path.name
            else:
                print(f"Warning: Image {file_name} not found (tried {src_path}).")
                continue

            # Create label file
            label_path = output_dir / 'labels' / split_name / f"{Path(file_name).stem}.txt"

            anns = img_to_anns.get(img_id, [])
            with open(label_path, 'w') as f:
                for ann in anns:
                    if 'segmentation' not in ann:
                        continue

                    cat_idx = cat_id_to_idx[ann['category_id']]

                    for seg in ann['segmentation']:
                        # seg is a list of coordinates [x1, y1, x2, y2, ...]
                        # Normalize
                        points = np.array(seg).reshape(-1, 2).astype(float)
                        points[:, 0] /= width
                        points[:, 1] /= height

                        # Clip to [0, 1] just in case
                        points = np.clip(points, 0, 1)

                        # Format: class x1 y1 x2 y2 ...
                        line = f"{cat_idx} " + " ".join([f"{x:.6f} {y:.6f}" for x, y in points])
                        f.write(line + "\n")

    process_images(train_images, 'train')
    process_images(val_images, 'val')

    # Create data.yaml
    yaml_content = f"""
path: {output_dir.absolute()}
train: images/train
val: images/val
names:
"""
    for idx, name in enumerate(cat_names):
        yaml_content += f"  {idx}: {name}\n"

    with open(output_dir / 'data.yaml', 'w') as f:
        f.write(yaml_content)

    print(f"Dataset prepared at {output_dir}")
    print(f"data.yaml created at {output_dir / 'data.yaml'}")

convert_coco_to_yolo(
    json_path=f'{DATA_DIR}/annotations.coco.json',
    source_img_dir=DATA_DIR,
    output_dir='datasets/contrail-seg'
)


Categories mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
Category names: ['contrail', 'contrail maybe', 'contrail old', 'contrail veryold', 'contrail young', 'parasite', 'sun', 'unknow']


Processing val: 100%|██████████| 320/320 [00:00<00:00, 372.62it/s]

Dataset prepared at datasets/contrail-seg
data.yaml created at datasets/contrail-seg/data.yaml


In [ ]:

# Custom callback to log metrics to W&B


In [22]:
with wandb.init(project="contrails-seg") as run:
    def on_train_epoch_end(trainer):
        """Callback that runs at the end of each training epoch."""
        metrics = trainer.metrics
        epoch = trainer.epoch

        # Log training metrics
        log_dict = {
            "epoch": epoch,
        }

        # Add box loss metrics
        if "train/box_loss" in metrics:
            log_dict["train/box_loss"] = metrics["train/box_loss"]
        if "train/seg_loss" in metrics:
            log_dict["train/seg_loss"] = metrics["train/seg_loss"]
        if "train/cls_loss" in metrics:
            log_dict["train/cls_loss"] = metrics["train/cls_loss"]
        if "train/dfl_loss" in metrics:
            log_dict["train/dfl_loss"] = metrics["train/dfl_loss"]

        # Add validation metrics if available
        if "metrics/precision(M)" in metrics:
            log_dict["val/precision"] = metrics["metrics/precision(M)"]
        if "metrics/recall(M)" in metrics:
            log_dict["val/recall"] = metrics["metrics/recall(M)"]
        if "metrics/mAP50(M)" in metrics:
            log_dict["val/mAP50"] = metrics["metrics/mAP50(M)"]
        if "metrics/mAP50-95(M)" in metrics:
            log_dict["val/mAP50-95"] = metrics["metrics/mAP50-95(M)"]

        # Log to W&B
        run.log(log_dict)
    def on_val_end(trainer):
        """Callback that runs at the end of validation."""
        metrics = trainer.metrics

        # Additional validation metrics can be logged here
        val_log_dict = {}

        if hasattr(metrics, 'box'):
            if hasattr(metrics.box, 'map'):
                val_log_dict["val/box_mAP"] = metrics.box.map
            if hasattr(metrics.box, 'map50'):
                val_log_dict["val/box_mAP50"] = metrics.box.map50

        if hasattr(metrics, 'seg'):
            if hasattr(metrics.seg, 'map'):
                val_log_dict["val/seg_mAP"] = metrics.seg.map
            if hasattr(metrics.seg, 'map50'):
                val_log_dict["val/seg_mAP50"] = metrics.seg.map50

        if val_log_dict:
            run.log(val_log_dict)
    model = YOLO("yolo11l-seg.pt")
    model.add_callback("on_train_epoch_end", on_train_epoch_end)
    model.add_callback("on_val_end", on_val_end)
    model.train(
        data=f"/content/datasets/contrail-seg/data.yaml",
        epochs=100,
        imgsz=640,
        name="contrail-segmentation-runL",
        exist_ok=True,
        batch=-1,
        save_period=5,
        project="/content/drive/MyDrive/contrails_seg/train2",

    )

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/contrail-seg/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=contrail-segmentation-runL, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

Traceback (most recent call last):
  File "/tmp/ipython-input-2754784786.py", line 59, in <cell line: 0>
    model.train(
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 773, in train
    self.trainer.train()
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py", line 243, in train
    self._do_train()
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py", line 478, in _do_train
    self.metrics, self.fitness = self.validate()
                                 ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py", line 704, in validate
    metrics = self.validator(self)
              ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/validator.py", line 237

epoch,▁
val/mAP50,▁
val/mAP50-95,▁
val/precision,▁
val/recall,▁
epoch,0
val/mAP50,0
val/mAP50-95,0
val/precision,0
val/recall,0


AttributeError: 'SegmentationValidator' object has no attribute 'validator'